In [ ]:
import tensorflow as tf
import keras
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Estrutura do AutoEncoder 
class AutoEncoder(tf.keras.Model):
    def __init__(self, latent_dim, im_size=(64, 64), channels=3, **kwargs):
        super(AutoEncoder, self).__init__(**kwargs)
        self.encoder = tf.keras.Sequential([
            keras.layers.Input(im_size + (channels,)),
            keras.layers.Conv2D(256, 3, padding='same'),
            keras.layers.LeakyReLU(),
            
            keras.layers.Conv2D(128, 3, padding='same'),
            keras.layers.LeakyReLU(),
            
            keras.layers.Conv2D(latent_dim, 3, padding='same', activation='sigmoid'),
        ])
        
        self.decoder = tf.keras.Sequential([
            keras.layers.Conv2DTranspose(latent_dim, 3, padding='same'),
            keras.layers.LeakyReLU(),
            
            keras.layers.Conv2DTranspose(128, 3, padding='same'),
            keras.layers.LeakyReLU(),            
            
            keras.layers.Conv2DTranspose(3, 3, padding='same', activation='sigmoid'),
        ])
    
    def call(self, inputs):
        z = self.encoder(inputs)
        return self.decoder(z)

In [ ]:
ae = AutoEncoder(latent_dim=64, im_size=(120, 120))
ae.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
def create_dataset(path):
    """função para pegar as imagens do diretório especificado
    e retornar em formato ndarray em escala de 0 e 1"""
    images = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            image = cv2.imread(os.path.join(dirname, filename))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image.astype('float32')
            image /= 255.0
            images.append(image)
    images = np.array(images)
    return images


faces_1 = create_dataset('dataset/face_a/')
faces_2 = create_dataset('dataset/face_b/')

In [ ]:
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(faces_1, faces_1, test_size=0.20, random_state=0)
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(faces_2, faces_2, test_size=0.15, random_state=0)

In [ ]:
# Treinamento para o rosto A
checkpoint_a = tf.keras.callbacks.ModelCheckpoint('./checkpoints/face_a', save_best_only=True)
ae.fit(X_train_a, X_train_a, epochs=1000, validation_data=(X_test_a, X_test_a), batch_size=32, callbacks=[checkpoint_a])

In [ ]:
# Treinamento para o rosto B
checkpoint_b = tf.keras.callbacks.ModelCheckpoint('./checkpoints/face_b', save_best_only=True)
ae.fit(X_train_b, X_train_b, epochs=1000, validation_data=(X_test_b, X_test_b), batch_size=32, callbacks=[checkpoint_b])

In [ ]:
# carregando os modelos salvos com os melhores pesos durante o treinamento
autoencoder_a = keras.models.load_model('./checkpoints/face_a')
autoencoder_b = keras.models.load_model('./checkpoints/face_b')

In [ ]:
# pegndo apenas o encoder treinado com o rosto A e o decoder treinado apenas com o rosto B
encoder_a = keras.Model(autoencoder_a.encoder.inputs, autoencoder_a.encoder.outputs)
decoder_b = keras.Model(autoencoder_b.decoder.inputs, autoencoder_b.decoder.outputs)

In [ ]:
# pegando dois rostos para testar
face_a = X_test_a[np.random.randint(X_test_a.shape[0])]
face_b = X_test_b[np.random.randint(X_test_b.shape[0])]
plt.subplot(1, 2, 1)
plt.imshow(face_a)
plt.title('face a')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(face_b)
plt.title('face b')
plt.axis('off')

In [ ]:
# fazendo as transformações
face_a_encoded = encoder_a.predict(np.expand_dims(face_a, 0))
face_b_decoded = decoder_b.predict(face_a_encoded)

In [ ]:
plt.title('Resultado')
plt.imshow(face_b_decoded.reshape(120, 120, 3))
plt.axis('off')